# **Project Name**    - Rossman Stores Sales Prediction



##### **Project Type**    - Regression
##### **Contribution**    - Team
##### **Team Member 1** - Nitin Pal
##### **Team Member 2** - Shristhi Patel


# **Project Summary -**

Rossman is based in European continent operating over 3000 drug stores. Currently they predict six weeks of sales in advance. The sales are influeneced by many factors like competition distance, promotions running, school, state holiday and locality. In this project we build a model based on the given data to predict the sales of the stores and get some insights like on which day store must be open for better sales.

# **GitHub Link -**

https://github.com/palnitin12345/Rossman-Sales-Prediction.git

# **Problem Statement**


**Predicting sales of Rossman stores**

# Main code

## **1. Know Your Data**

In [ ]:
#Importing libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from numpy import math
from scipy import stats

import datetime
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import requests
import io

In [ ]:
# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/palnitin12345/Rossman-Sales-Prediction/main/Rossmann_Stores_Data.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

url1 = "https://raw.githubusercontent.com/palnitin12345/Rossman-Sales-Prediction/main/store.csv" # Make sure the url is the raw version of the file on GitHub
download1 = requests.get(url1).content

# Reading the downloaded content and turning it into a pandas dataframe

data1 = pd.read_csv(io.StringIO(download.decode('utf-8')))
data2 = pd.read_csv(io.StringIO(download1.decode('utf-8')))

In [ ]:
#First look of the data.
data1.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [ ]:
data2.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [ ]:
# Dataset Rows & Columns count

print(f'Shape of first dataset {data1.shape}')
print(f'Shape of second dataset {data2.shape}')

Shape of first dataset (1017209, 9)
Shape of second dataset (1115, 10)


In [ ]:
#Merging both the dataframes based on Store id.

dataset = data1.merge(data2, on = 'Store',how = 'left')

In [ ]:
print(f'Shape of second dataset {dataset.shape}')

Shape of second dataset (1017209, 18)


In [ ]:
#Looking the datatypes

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

Duplicate Values

In [ ]:

#Number of duplicated Rows

len(dataset[dataset.duplicated()])

0

Missing/Null Values

In [ ]:
#Finding null columns in the dataset and percentage of null values.
print("Col     Null values percent")
for col in dataset.columns:
  if dataset[col].notnull().sum() != len(dataset):
    print(f"{col}  :- {round(dataset[col].isnull().sum()*100/len(dataset),2)}")

Col     Null values percent
CompetitionDistance  :- 0.26
CompetitionOpenSinceMonth  :- 31.79
CompetitionOpenSinceYear  :- 31.79
Promo2SinceWeek  :- 49.94
Promo2SinceYear  :- 49.94
PromoInterval  :- 49.94


### What did you know about your dataset?

## **2. Understanding the variables**

In [ ]:
# Dataset Columns

dataset.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [ ]:
# Dataset Describe

dataset.describe(include='all')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1017209,1017209,1.014567e+06,693861.000000,693861.000000,1.017209e+06,509178.000000,509178.000000,509178
unique,NaN,NaN,942,NaN,NaN,NaN,NaN,5,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,NaN,2015-07-31,NaN,NaN,NaN,NaN,0,NaN,a,a,NaN,NaN,NaN,NaN,NaN,NaN,"Jan,Apr,Jul,Oct"
freq,NaN,NaN,1115,NaN,NaN,NaN,NaN,855087,NaN,551627,537445,NaN,NaN,NaN,NaN,NaN,NaN,293122
mean,5.584297e+02,3.998341e+00,NaN,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,NaN,1.786467e-01,NaN,NaN,5.430086e+03,7.222866,2008.690228,5.005638e-01,23.269093,2011.752774,NaN
std,3.219087e+02,1.997391e+00,NaN,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,NaN,3.830564e-01,NaN,NaN,7.715324e+03,3.211832,5.992644,4.999999e-01,14.095973,1.662870,NaN
min,1.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,2.000000e+01,1.000000,1900.000000,0.000000e+00,1.000000,2009.000000,NaN
25%,2.800000e+02,2.000000e+00,NaN,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,7.100000e+02,4.000000,2006.000000,0.000000e+00,13.000000,2011.000000,NaN
50%,5.580000e+02,4.000000e+00,NaN,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,2.330000e+03,8.000000,2010.000000,1.000000e+00,22.000000,2012.000000,NaN
75%,8.380000e+02,6.000000e+00,NaN,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,NaN,0.000000e+00,NaN,NaN,6.890000e+03,10.000000,2013.000000,1.000000e+00,37.000000,2013.000000,NaN


Variables Description 

1. Store           : unique ID

2. Open            : 0 = store closed, 1 = store open

3. StateHoliday    : holiday. a = public holiday, b = Easter holiday, c =       Christmas, 0 = None

4. SchoolHoliday   : the closure of public schools

5. StoreType       : a, b, c, d

6. Assortment      : assortment level, a = basic, b = extra, c = extended

7. Promo           : store is running a promo on that day

8. Promo2          : 0 = store not participating, 1 = store is participating

9. CompetitionOpenSince[Month/Year]  : approximate year and month of the time, the nearest competitor store was opened

10. Promo2Since[Year/Week]            : describes the year and calendar week when the store started participating in Promo2

11. PromoInterval   : describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew.

12. DayOfWeek       : Day of the week, using 1-7 for Mon-Sat

13. Date            : Date in YYYY-MM-DD


14. Sales           : the number of sales of the store that day

15. Customers       : number of customers

16. CompetitionDistance :   distance to the nearest competitor store



Unique Values

In [ ]:
# Check Unique Values for each variable.
for i in dataset.columns.tolist():
  print("No. of unique values in ",i,"is",dataset[i].nunique(),)

No. of unique values in  Store is 1115
No. of unique values in  DayOfWeek is 7
No. of unique values in  Date is 942
No. of unique values in  Sales is 21734
No. of unique values in  Customers is 4086
No. of unique values in  Open is 2
No. of unique values in  Promo is 2
No. of unique values in  StateHoliday is 5
No. of unique values in  SchoolHoliday is 2
No. of unique values in  StoreType is 4
No. of unique values in  Assortment is 3
No. of unique values in  CompetitionDistance is 654
No. of unique values in  CompetitionOpenSinceMonth is 12
No. of unique values in  CompetitionOpenSinceYear is 23
No. of unique values in  Promo2 is 2
No. of unique values in  Promo2SinceWeek is 24
No. of unique values in  Promo2SinceYear is 7
No. of unique values in  PromoInterval is 3


In [ ]:
#Columns that are categorical in nature

for col in dataset.columns:
  if dataset[col].nunique()<=10:
    print(col)
    print(dataset[col].value_counts())
    print('')

DayOfWeek
5    145845
4    145845
3    145665
2    145664
1    144730
7    144730
6    144730
Name: DayOfWeek, dtype: int64

Open
1    844392
0    172817
Name: Open, dtype: int64

Promo
0    629129
1    388080
Name: Promo, dtype: int64

StateHoliday
0    855087
0    131072
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

SchoolHoliday
0    835488
1    181721
Name: SchoolHoliday, dtype: int64

StoreType
a    551627
d    312912
c    136840
b     15830
Name: StoreType, dtype: int64

Assortment
a    537445
c    471470
b      8294
Name: Assortment, dtype: int64

Promo2
1    509178
0    508031
Name: Promo2, dtype: int64

Promo2SinceYear
2011.0    115056
2013.0    110464
2014.0     79922
2012.0     73174
2009.0     65270
2010.0     56240
2015.0      9052
Name: Promo2SinceYear, dtype: int64

PromoInterval
Jan,Apr,Jul,Oct     293122
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: PromoInterval, dtype: int64



## ***3. Data Wrangling***

In [ ]:
#State Holiday column has some datatype errors, correcting them.

dataset['StateHoliday'].replace('0','d', inplace=True)
dataset['StateHoliday'].replace(0,'d', inplace=True)


#### Handling Null/Missing values

**Competition Distance**

In [ ]:
#%age of null values in Competition Distance column is

dataset['CompetitionDistance'].isnull().sum()*100/len(dataset['CompetitionDistance'])

#Treating Null Values

#Column CompetitionDistance has some null values, so replacing those null values with median of the columnn

dataset['CompetitionDistance'].fillna(dataset['CompetitionDistance'].median(), inplace=True)


**Competition Open Since Year and Month**

In [ ]:
#Column CompetitionOpenSinceMonth and CompetitionOpenSinceYear has some null values, so replacing those null values with mode of the respective columnns.


dataset['CompetitionOpenSinceMonth'].fillna(dataset['CompetitionOpenSinceMonth'].mode()[0],inplace=True)


dataset['CompetitionOpenSinceYear'].fillna(dataset['CompetitionOpenSinceYear'].mode()[0],inplace=True)

In [ ]:
#Making a new column,Competition Age with zero values, which will store the years of Competition running.
dataset['Competition_Age'] = 0

#Converting the datatype to int.
dataset['CompetitionOpenSinceMonth'] = dataset['CompetitionOpenSinceMonth'].astype(int)
dataset['CompetitionOpenSinceYear'] = dataset['CompetitionOpenSinceYear'].astype(int)

#making the reference timestamp of now.
now = pd.Timestamp('now')

#Merging the two columns to get the date like format
dataset['Competition_temp'] = dataset['CompetitionOpenSinceMonth'].astype(str) + "/" + dataset['CompetitionOpenSinceYear'].astype(str)

#Converting to datetime
dataset['Competition_temp'] = pd.to_datetime(dataset['Competition_temp'])

#getting the age of the competition.
dataset['Competition_Age'] = (now - dataset['Competition_temp']).astype('<m8[Y]')

#Dropping the unnecessary columns.
dataset.drop(['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Competition_temp'],axis=1,inplace=True)

**Promo2**

In [ ]:
#making a function that takes a week number and results the month number.

def week_to_month(t_week):
  t_year = 2000              
  date = str(t_year) + '-W' + str(t_week)
  
  # getting date
  res = datetime.strptime(date + '-1', "%Y-W%W-%w")

  return res.month

In [ ]:
#Now what I want to do is merge three columns Promo2 ones...by calculating the age since when the promo2 started,,, comparing with the sales..and age 0 will imply the store didn't run promo2.

#Making a new column with zero values, Promo2_age, which will store for how much time, the store has been running promotion.
dataset['Promo2_Age'] = 0

#To avoid Valueerrors, filling empty places with 0
dataset['Promo2SinceWeek'].fillna(0, inplace=True)
dataset['Promo2SinceYear'].fillna(0, inplace=True)

#Since both Promo2SinceWeek and Promo2SinceYear are datetime objects so, converting them to int from float for easy calculations.
dataset['Promo2SinceWeek'] = dataset['Promo2SinceWeek'].astype(int)
dataset['Promo2SinceYear'] = dataset['Promo2SinceYear'].astype(int)

#Applying the function to convert weeks to months.
dataset['Promo2SinceMonth'] = dataset['Promo2SinceWeek'].apply(lambda x: 0 if x==0 else week_to_month(x))

#Combining both weeeks and years columns to get a date.
dataset['Promo2_temp'] = dataset['Promo2SinceMonth'].astype(str) + "/" +dataset['Promo2SinceYear'].astype(str)

#Replacing 0/0 values to 0.
dataset['Promo2_temp'].replace('0/0',0, inplace=True)

#Converting to datetime for calculating the age.
dataset['Promo2_temp'] = pd.to_datetime(dataset['Promo2_temp'])

#Calculating the age of Promo2 and saving the year only.
dataset['Promo2_Age'] = (now - dataset['Promo2_temp']).astype('<m8[Y]') 

#Beacuse of null values some values showed 53, so replacing those with 0
dataset['Promo2_Age'].replace(53.0,0,inplace=True)

#Dropping the unecessary columns.
dataset.drop(['Promo2SinceMonth','Promo2SinceYear', 'Promo2SinceWeek','Promo2_temp'],axis=1, inplace=True)

**Date**

In [ ]:
#Here date is object-type, changing it in datetime type andn expanding columns.
dataset["Date"] =  pd.to_datetime(dataset["Date"], format="%Y/%m/%d")

#Extracting month, quarter and year from the date
 
dataset['Month'] = dataset['Date'].dt.month
dataset['Year'] = dataset['Date'].dt.year
dataset['Quarter'] = dataset['Date'].dt.quarter 